<a href="https://colab.research.google.com/github/neohack22/ML-engineering/blob/master/00-getting-started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Getting started with the Converse API in Amazon Bedrock

> *This notebook should work well with the **`Python 3`** kernel in SageMaker Studio*

In this notebook, we'll explore the basics of the Converse API in Amazon Bedrock. The Converse or ConverseStream API is a unified structured text API action that allows you simplifying the invocations to Bedrock LLMs, using a universal syntax and message structured prompts for any of the supported model providers.

Let's start by installing or updating boto3. You just need to run this cell the first time.

In [ ]:
%pip install --force-reinstall -q -r ./utils/requirements.txt

In [ ]:
import boto3
import sys
print('Running boto3 version:', boto3.__version__)

Running boto3 version: 1.35.13


Let's define the region and models to use. We can also setup our boto3 client.

In [ ]:
boto3_session = boto3.session.Session()
region = boto3_session.region_name

print('Using region: ', region)

bedrock = boto3.client(
    service_name = 'bedrock-runtime',
    region_name = region,
    )

MODEL_IDS = [
    "amazon.titan-tg1-large",
    "anthropic.claude-3-haiku-20240307-v1:0",
    "anthropic.claude-3-sonnet-20240229-v1:0",
]

Using region:  us-west-2


## Utilisation de la session boto3 et du client Bedrock Runtime

Ce code Python utilise la librairie boto3 pour interagir avec le service Amazon SageMaker Bedrock Runtime.

* **Configuration de la session boto3**
  - `boto3_session = boto3.session.Session()`: Cette ligne crée une session boto3. La session gère les informations d'identification et la région AWS par défaut  [➊](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/session.html).
  - `region = boto3_session.region_name`: Cette ligne récupère la région AWS configurée par défaut dans la session boto3.

* **Création du client Bedrock Runtime**
  - `bedrock = boto3.client(service_name='bedrock-runtime', region_name=region)`: Cette ligne crée un client pour interagir avec le service Bedrock Runtime. On précise le nom du service (`bedrock-runtime`) et la région à utiliser (`region`).

* **Liste des modèles**
  - `MODEL_IDS = [...]`: Cette liste contient les identifiants de plusieurs modèles Bedrock que l'on souhaite peut-être utiliser plus tard dans le code.

We're now ready to setup our Converse API action in Bedrock. Note that we use the same syntax for any model, including the messages-formatted prompts, and the inference parameters. Also note that we read the output in the same way independently of the model used.

Optionally, we could define additional model specific request fields that are not common across all providers. For more information on this check the Bedrock Converse API documentation.

### Converse for one-shot invocations

In [ ]:
def invoke_bedrock_model(client, id, prompt, max_tokens=2000, temperature=0, top_p=0.9):
    response = ""
    try:
        response = client.converse(
            modelId=id,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "text": prompt
                        }
                    ]
                }
            ],
            inferenceConfig={
                "temperature": temperature,
                "maxTokens": max_tokens,
                "topP": top_p
            }
            #additionalModelRequestFields={
            #}
        )
    except Exception as e:
        print(e)
        result = "Model invocation error"
    try:
        result = response['output']['message']['content'][0]['text'] \
        + '\n--- Latency: ' + str(response['metrics']['latencyMs']) \
        + 'ms - Input tokens:' + str(response['usage']['inputTokens']) \
        + ' - Output tokens:' + str(response['usage']['outputTokens']) + ' ---\n'
        return result
    except Exception as e:
        print(e)
        result = "Output parsing error"
    return result

## Analyse de la fonction `invoke_bedrock_model`

**Fonctionnalité:**

La fonction `invoke_bedrock_model` est conçue pour appeler un modèle Amazon SageMaker Bedrock et renvoyer le résultat de l'invocation, ainsi que des métriques sur la latence et l'utilisation des tokens.

**Paramètres:**

- `client`: Un objet client boto3 pour interagir avec le service Bedrock Runtime.
- `id`: L'identifiant du modèle Bedrock à invoquer.
- `prompt`: L'invite ou la question à poser au modèle.
- `max_tokens`: Le nombre maximum de tokens à générer dans la réponse.
- `temperature`: Un paramètre de contrôle de la randomness de la génération.
- `top_p`: Un autre paramètre de contrôle de la randomness de la génération.

**Fonctionnement:**

1. **Invocation du modèle:**
   - La fonction appelle la méthode `converse` du client Bedrock Runtime pour invoquer le modèle spécifié avec l'invite donnée.
   - Les paramètres `inferenceConfig` contrôlent la génération de la réponse (nombre maximum de tokens, randomness).

2. **Gestion des erreurs:**
   - Si une erreur se produit lors de l'invocation du modèle, la fonction renvoie `"Model invocation error"`.

3. **Extraction et formatage du résultat:**
   - Si l'invocation réussit, la fonction extrait le texte de la réponse, ainsi que les métriques de latence et d'utilisation des tokens.
   - Le résultat est formaté sous la forme d'une chaîne de caractères.

4. **Gestion des erreurs de parsing:**
   - Si une erreur se produit lors du parsing du résultat, la fonction renvoie `"Output parsing error"`.

**Code snippet:**

```markdown
def invoke_bedrock_model(client, id, prompt, max_tokens=2000, temperature=0, top_p=0.9):
    # ... (corps de la fonction)
```

Finally, we can test our invocation.

In this example, we run the same prompt across all the text models supported in Bedrock by the time of writing this example.

In [ ]:
prompt = ("What is the capital of Italy?")
print(f'Prompt: {prompt}\n')

for i in MODEL_IDS:
    response = invoke_bedrock_model(bedrock, i, prompt)
    print(f'Model: {i}\n{response}')

Prompt: What is the capital of Italy?

Model: amazon.titan-tg1-large
The capital of Italy is Rome. It is the fourth most populous city in the European Union.
--- Latency: 1375ms - Input tokens:10 - Output tokens:24 ---

Model: anthropic.claude-3-haiku-20240307-v1:0
The capital of Italy is Rome.
--- Latency: 224ms - Input tokens:14 - Output tokens:10 ---

Model: anthropic.claude-3-sonnet-20240229-v1:0
The capital of Italy is Rome.
--- Latency: 283ms - Input tokens:14 - Output tokens:10 ---



1. Il définit une variable `prompt` contenant une question sur la capitale de l'Italie.

2. Il imprime cette question en utilisant une f-string.

3. Ensuite, il y a une boucle `for` qui itère sur une liste ou un itérable appelé `MODEL_IDS`. Pour chaque élément `i` dans `MODEL_IDS` :

   - Il appelle une fonction `invoke_bedrock_model` avec les paramètres `bedrock`, `i`, et `prompt`.
   - Il imprime le résultat de cette fonction, en précisant le modèle utilisé (`i`) et la réponse obtenue.

Voici comment on pourrait améliorer ce code en ajoutant des commentaires explicatifs :

```python
# Définition de la question
prompt = "What is the capital of Italy?"
print(f'Prompt: {prompt}\n')

# Itération sur les différents modèles
for i in MODEL_IDS:
    # Appel du modèle Bedrock avec la question
    response = invoke_bedrock_model(bedrock, i, prompt)
    
    # Affichage du résultat pour chaque modèle
    print(f'Model: {i}\n{response}')
```

utilisé pour tester différents modèles d'IA (probablement des modèles de traitement du langage naturel) avec une même question, et comparer leurs réponses.

### ConverseStream for streaming invocations

We can also use the Converse API for streaming invocations. In this case we rely on the ConverseStream action.

In [ ]:
MODEL_IDS = [
    "amazon.titan-tg1-large",
    "anthropic.claude-3-haiku-20240307-v1:0",
    "anthropic.claude-3-sonnet-20240229-v1:0",
]

éfinit une liste appelée `MODEL_IDS` contenant trois chaînes de caractères. Chaque chaîne représente un identifiant de modèle, pour des modèles d'intelligence artificielle ou de traitement du langage naturel.

Voici une explication plus détaillée :

1. La liste est définie à l'aide de crochets `[]`.
2. Chaque élément de la liste est une chaîne de caractères (string) délimitée par des guillemets doubles `"`.
3. Les éléments sont séparés par des virgules.

Pour utiliser cette liste dans votre code Python :

```python
# Accéder au premier élément de la liste
premier_modele = MODEL_IDS[0]
print(premier_modele)  # Affichera "amazon.titan-tg1-large"

# Parcourir tous les éléments de la liste
for modele in MODEL_IDS:
    print(modele)

# Obtenir la longueur de la liste
nombre_de_modeles = len(MODEL_IDS)
print(f"Il y a {nombre_de_modeles} modèles dans la liste.")
```

Cette structure de données est utile lorsque vous avez besoin de stocker et d'accéder à une collection ordonnée d'identifiants de modèles dans votre programme Python.

In [ ]:
def invoke_bedrock_model_stream(client, id, prompt, max_tokens=2000, temperature=0, top_p=0.9):
    response = ""
    response = client.converse_stream(
        modelId=id,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "text": prompt
                    }
                ]
            }
        ],
        inferenceConfig={
            "temperature": temperature,
            "maxTokens": max_tokens,
            "topP": top_p
        }
    )
    # Extract and print the response text in real-time.
    for event in response['stream']:
        if 'contentBlockDelta' in event:
            chunk = event['contentBlockDelta']
            sys.stdout.write(chunk['delta']['text'])
            sys.stdout.flush()
    return

Cette fonction `invoke_bedrock_model_stream` semble être conçue pour interagir avec un modèle de langage, probablement via l'API Bedrock d'Amazon Web Services. Voici les principales caractéristiques :

1. La fonction prend plusieurs paramètres :
   - `client` : probablement un objet client pour l'API Bedrock
   - `id` : l'identifiant du modèle à utiliser
   - `prompt` : le texte d'entrée pour le modèle
   - `max_tokens`, `temperature`, et `top_p` : des paramètres de configuration pour l'inférence

2. Elle utilise la méthode `converse_stream` du client pour envoyer une requête au modèle.

3. La réponse est traitée de manière streaming, ce qui signifie que la fonction reçoit et traite la réponse par morceaux au fur et à mesure qu'elle arrive.

4. Chaque morceau de texte reçu est immédiatement écrit sur la sortie standard (`sys.stdout`) et le buffer est vidé (`sys.stdout.flush()`), ce qui permet d'afficher la réponse en temps réel.

Voici un exemple de comment on pourrait utiliser cette fonction :

```python
import boto3

# Initialiser le client Bedrock
bedrock = boto3.client('bedrock-runtime')

# Utiliser la fonction
invoke_bedrock_model_stream(
    client=bedrock,
    id="votre_modele_id",
    prompt="Votre question ou prompt ici",
    max_tokens=1000,
    temperature=0.7
)
```

Cette fonction est utile pour des applications qui nécessitent une réponse en temps réel d'un modèle de langage, comme des chatbots ou des assistants IA interactifs.

In [ ]:
prompt = ("What is the capital of Italy?")
print(f'Prompt: {prompt}\n')

for i in MODEL_IDS:
    print(f'\n\nModel: {i}')
    invoke_bedrock_model_stream(bedrock, i, prompt)

Prompt: What is the capital of Italy?



Model: amazon.titan-tg1-large
The capital of Italy is Rome. It is the fourth most populous city in the European Union.

Model: anthropic.claude-3-haiku-20240307-v1:0
The capital of Italy is Rome.

Model: anthropic.claude-3-sonnet-20240229-v1:0
The capital of Italy is Rome.

1. Une variable `prompt` est définie avec une chaîne de caractères contenant une question.

2. Le prompt est affiché à l'aide de la fonction `print()` avec une f-string.

3. Une boucle `for` itère sur une liste ou un itérable appelé `MODEL_IDS`.

4. Pour chaque modèle dans `MODEL_IDS`, le code affiche le nom du modèle et appelle une fonction `invoke_bedrock_model_stream()`.

Pour améliorer ce code et le rendre plus robuste, voici quelques suggestions :

```python
MODEL_IDS = ["model1", "model2", "model3"]  # Définir la liste des modèles

prompt = "Quelle est la capitale de l'Italie ?"
print(f'Prompt : {prompt}\n')

for model_id in MODEL_IDS:
    print(f'\nModèle : {model_id}')
    try:
        response = invoke_bedrock_model_stream(bedrock, model_id, prompt)
        print(f'Réponse : {response}')
    except Exception as e:
        print(f'Erreur lors de l\'invocation du modèle : {e}')
```

Ce code amélioré :
1. Définit explicitement `MODEL_IDS`.
2. Utilise un nom de variable plus descriptif dans la boucle.
3. Ajoute une gestion des erreurs pour traiter les problèmes potentiels lors de l'appel à `invoke_bedrock_model_stream()`.
4. Suppose que `invoke_bedrock_model_stream()` renvoie une réponse et l'affiche.

N'oubliez pas d'importer les modules nécessaires et de configurer correctement l'objet `bedrock` avant d'exécuter ce code.

As you can see, the Converse API allow us to easily run the invocations with the same syntax across all the models.